In [343]:
import pandas as pd
import numpy as np

In [344]:
#Import Cities Data
cities_2018_disclosure = pd.read_csv('../raw_data/Cities/Cities Disclosing/2018_Cities_Disclosing_to_CDP.csv')
cities_2019_disclosure = pd.read_csv('../raw_data/Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv')
cities_2020_disclosure = pd.read_csv('../raw_data/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv')

In [345]:
#combine files
cities_disclosure=[cities_2018_disclosure,cities_2019_disclosure,cities_2020_disclosure]
cities_disclosure= pd.concat(cities_disclosure).sort_values('Account Number').reset_index(drop=True)
cities_disclosure.head()

#cities_disclosure.shape (20168,13)

,Year Reported to CDP,Account Number,Organization,City,Country,CDP Region,Reporting Authority,Access,First Time Discloser,Population,Population Year,City Location,Last update
0,2019,1093,City of Atlanta,Atlanta,United States of America,North America,"CDP Cities,ICLEI - Local Governments for Susta...",public,No,450182.0,2017,POINT (-84.388 33.749),2020-06-25T04:52:51.327
1,2018,1093,City of Atlanta,Atlanta,United States of America,North America,CDP Cities,public,No,450182.0,2017,POINT (-84.388 33.749),2020-06-25T04:52:49.050
2,2018,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities",public,No,1265974.0,2017,POINT (-97.7431 30.2672),2020-06-25T04:52:49.050
3,2020,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,964254.0,2019,POINT (-97.7431 30.2672),2020-09-07T09:45:19.793
4,2019,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,1227000.0,2017,POINT (-97.7431 30.2672),2020-06-25T04:52:51.327


In [346]:
#Adjust column names
cities_disclosure.columns=['Reporting_Year','Account_Num','Org','City','Country','CDP_Reg','Reporting_Auth',\
                          'Access','First_Time_Disc','Pop','Pop_Year','City_Loc','Last_Update']

## Dealing with Nulls

### City

In [347]:
#Lets look at nulls
cities_disclosure.isnull().sum()

Reporting_Year       0
Account_Num          0
Org                  0
City               352
Country              0
CDP_Reg              0
Reporting_Auth       0
Access               0
First_Time_Disc      0
Pop                284
Pop_Year             0
City_Loc           438
Last_Update          0
dtype: int64

The dataset contains data from both inside and outside of the US. It may be more common for countries to not list a city (since they may not use cities). I'll set non-US nulls to "Not Listed" and remove US nulls if the number of missing data is less than 5%.

In [348]:
#Nulls from outside the US
cities_disclosure.loc[(cities_disclosure.City.isnull())&\
                      (cities_disclosure.Country!='United States of America'),'City']='Not Listed'

In [349]:
num = cities_disclosure.loc[(cities_disclosure.City.isnull())&\
                      (cities_disclosure.Country=='United States of America')].shape[0]
denom = cities_disclosure['City'].shape[0]

print('Total percentage of missing nulls are {}%'.format(round((num/denom)*100,2)))

Total percentage of missing nulls are 2.03%


### Pop

Find an example where We have populations for some dates but not others

In [350]:
cities_disclosure.loc[cities_disclosure['Account_Num']==10894]

,Reporting_Year,Account_Num,Org,City,Country,CDP_Reg,Reporting_Auth,Access,First_Time_Disc,Pop,Pop_Year,City_Loc,Last_Update
33,2018,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities",non public,No,4030904.0,2016,POINT (-118.244 34.0522),2020-06-25T04:52:49.050
34,2020,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,4021488.0,2018,POINT (-118.244 34.0522),2020-09-07T09:45:19.793
35,2019,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,NaN,0,POINT (-118.244 34.0522),2020-06-25T04:52:51.327


In [351]:
(2016+2018)/2

2017.0

In [352]:
cities_disclosure['Pop'] = \
cities_disclosure.groupby(['Account_Num'],sort=False)['Pop'].apply(lambda x: x.fillna(x.mean()))

In [353]:
assert cities_disclosure.loc[35,'Pop']==4026196.0

Replace pop_year with average year (to match us taking the average population value)

In [354]:
#replace 0s with nulls
cities_disclosure['Pop_Year'] = \
cities_disclosure.groupby(['Account_Num'])['Pop_Year'].apply(lambda x: x.replace(0,np.nan))


#replace nulls with averag value

cities_disclosure['Pop_Year'] = \
cities_disclosure.groupby(['Account_Num'])['Pop_Year'].apply(lambda x: x.fillna(x.mean()))

In [355]:
assert cities_disclosure.loc[35,'Pop_Year']==2017

In [356]:
cities_disclosure.Pop_Year.unique()

array([2017. , 2019. , 2018. , 2015. , 2016. , 2020. , 2012. , 2011. ,
       2017.5, 2013. , 2010. , 2014. , 2016.5,    nan, 2018.5, 2013.5,
          7. , 1013. , 2007. ,  214. , 2005. , 2012.5, 2019.5,  216. ,
        217. ,   19. , 2003. , 2014.5, 2009. , 2008. ,  218. ,  219. ,
         21. , 1537. ,  211. , 2002. ])

Deal with these later. i dont think it matters

In [357]:
Cities_2018_Responses = pd.read_csv('../raw_data/Cities/Cities Responses/2018_Full_Cities_Dataset.csv')
Cities_2019_Responses = pd.read_csv('../raw_data/Cities/Cities Responses/2019_Full_Cities_Dataset.csv')
Cities_2020_Responses = pd.read_csv('../raw_data/Cities/Cities Responses/2020_Full_Cities_Dataset.csv')

Cities_Responses=pd.concat([Cities_2018_Responses,Cities_2019_Responses,Cities_2020_Responses]).sort_values('Account Number').reset_index(drop=True)
Cities_Responses.head()

,Questionnaire,Year Reported to CDP,Account Number,Organization,Country,CDP Region,Parent Section,Section,Question Number,Question Name,Column Number,Column Name,Row Number,Row Name,Response Answer,Comments,File Name,Last update
0,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Opportunities,Opportunities,6.2,"List any emission reduction, adaptation, water...",5,Project description,2,NaN,Renew Atlanta is an infrastructure improvement...,NaN,NaN,24/06/2020 05:30:36 AM
1,CDP Cities 2018,2018,1093,City of Atlanta,United States of America,North America,City Wide Emissions,GHG Emissions Data,7.5,Please attach your city-wide inventory in the ...,4,Comment on level of confidence,1,NaN,We were unable to retrieve natural gas data fr...,NaN,NaN,24/06/2020 05:28:18 AM
2,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,12,Please indicate to which sector(s) the target ...,1,NaN,Public facility,NaN,NaN,24/06/2020 05:30:36 AM
3,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,9,Target year absolute emissions (metric tonnes ...,1,NaN,NaN,NaN,NaN,24/06/2020 05:30:36 AM
4,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Climate Hazards & Vulnerability,Climate Hazards,2.2,Please identify and describe the factors that ...,2,Support / Challenge,4,NaN,Challenge,NaN,NaN,24/06/2020 05:30:36 AM


In [358]:
Cities_Responses.drop(columns=['Questionnaire','Organization','Country','CDP Region',\
                               'File Name','Last update','Comments'],inplace=True)

In [359]:
Cities_Responses.columns=['Year','Account_Num','Parent_Sect','Sect','Q_Num','Q_Name',\
                          'Col_Num','Col_Name','Row_Num','Row_Name','Answer']
Cities_Responses.head()

,Year,Account_Num,Parent_Sect,Sect,Q_Num,Q_Name,Col_Num,Col_Name,Row_Num,Row_Name,Answer
0,2019,1093,Opportunities,Opportunities,6.2,"List any emission reduction, adaptation, water...",5,Project description,2,NaN,Renew Atlanta is an infrastructure improvement...
1,2018,1093,City Wide Emissions,GHG Emissions Data,7.5,Please attach your city-wide inventory in the ...,4,Comment on level of confidence,1,NaN,We were unable to retrieve natural gas data fr...
2,2019,1093,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,12,Please indicate to which sector(s) the target ...,1,NaN,Public facility
3,2019,1093,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,9,Target year absolute emissions (metric tonnes ...,1,NaN,NaN
4,2019,1093,Climate Hazards & Vulnerability,Climate Hazards,2.2,Please identify and describe the factors that ...,2,Support / Challenge,4,NaN,Challenge


In [360]:
#Create table of unique questions from the questionaire
Questionaire_Questions =Cities_Responses[['Year','Parent_Sect','Sect','Q_Num',\
                                          'Q_Name','Col_Num','Col_Name','Row_Num','Row_Name']].drop_duplicates()
Questionaire_Questions=Questionaire_Questions.sort_values(['Q_Num','Col_Num','Row_Num','Year']).reset_index(drop=True)
Questionaire_Questions.index=Questionaire_Questions.index.set_names(['Question_ID'])
Questionaire_Questions.reset_index(inplace=True)
Questionaire_Questions.head()

,Question_ID,Year,Parent_Sect,Sect,Q_Num,Q_Name,Col_Num,Col_Name,Row_Num,Row_Name
0,0,2018,NaN,Introduction,0.1,Please give a general description and introduc...,1,Administrative boundary,1,City boundary
1,1,2019,NaN,Introduction,0.1,Please give a general description and introduc...,1,Administrative boundary,1,City boundary
2,2,2020,NaN,Introduction,0.1,Please give a general description and introduc...,1,Administrative boundary,1,Please complete
3,3,2018,NaN,Introduction,0.1,Please give a general description and introduc...,2,Description of city,1,City boundary
4,4,2019,NaN,Introduction,0.1,Please give a general description and introduc...,2,Description of city,1,City boundary


In [361]:
df = Cities_Responses.merge(Questionaire_Questions,on=['Year','Q_Num','Q_Name','Col_Num','Col_Name','Row_Num','Row_Name'])[['Account_Num','Year','Question_ID','Answer']]
df.sort_values('Account_Num')

,Account_Num,Year,Question_ID,Answer
0,1093,2019,18980,Renew Atlanta is an infrastructure improvement...
290743,1093,2019,7402,Extreme Precipitation > Rain storm
291378,1093,2018,9600,Rain storm
291671,1093,2019,2380,NaN
292485,1093,2018,20668,2020
...,...,...,...,...
575441,848478,2020,2441,Question not applicable
530785,848478,2020,18845,Question not applicable
780519,848478,2020,2183,Question not applicable
1051979,848478,2020,13157,Question not applicable


Lets test to make sure that the question_ID corresponds to the correct Answe. This test will:
 1. 

In [362]:
def test_merge_Q_id(account_Num,Question_ID,answer):
    '''
    This is to make sure that I lined up the Questionare_Questions DF to the Cities_Response DataFrame.
    Enter an account_Num, Question_ID, and the answer that you are expecting
    '''
    

    qq=Questionaire_Questions.loc[Questionaire_Questions.Question_ID==Question_ID] 
    cr=Cities_Responses.loc[Cities_Responses['Account_Num']==account_Num]\
    [['Answer','Year','Q_Num','Q_Name','Col_Num','Col_Name','Row_Num','Row_Name']]
    result = cr.loc[(cr.Year.isin(qq.Year))&(cr.Q_Num.isin(qq.Q_Num))&(cr.Col_Num.isin(qq.Col_Num))&(cr.Row_Num.isin(qq.Row_Num)),'Answer']
    assert answer == result.values


In [363]:
test_merge_Q_id(1093,7402,'Extreme Precipitation > Rain storm')
test_merge_Q_id(848478,18845,'Question not applicable')

All good! No errors

## Cleaning up the df

In [364]:
df.dtypes

Account_Num     int64
Year            int64
Question_ID     int64
Answer         object
dtype: object

In [365]:
#Set to category for one hot encoding 
df.Year = df.Year.astype('category')
df.Question_ID = df.Question_ID.astype('category')

Check for nulls

In [366]:
df.isnull().sum()

Account_Num         0
Year                0
Question_ID         0
Answer         401188
dtype: int64

In [367]:
df.loc[df.Answer.isna()]

,Account_Num,Year,Question_ID,Answer
21,31174,2019,18980,NaN
42,35873,2019,18980,NaN
55,36036,2019,18980,NaN
76,44185,2019,18980,NaN
77,45219,2019,18980,NaN
...,...,...,...,...
1542460,74423,2018,20102,NaN
1542461,74423,2018,20108,NaN
1542462,74423,2018,20114,NaN
1542463,74423,2018,20096,NaN


we can remove these from our df since they are blanks

In [368]:
df.dropna(inplace=True)

In [369]:
df.head()

,Account_Num,Year,Question_ID,Answer
0,1093,2019,18980,Renew Atlanta is an infrastructure improvement...
1,1499,2019,18980,Barcelona City Council is working to promote a...
2,2430,2019,18980,"In order to achieve net zero energy status, at..."
3,3422,2019,18980,Mayor's energy efficiency programmes for both ...
4,10495,2019,18980,"In 2011, The City of Las Vegas selected stree..."


In [370]:
cities_disclosure.City_Loc.unique()

array(['POINT (-84.388 33.749)', 'POINT (-97.7431 30.2672)',
       'POINT (2.1775 41.3823)', nan, 'POINT (-73.2121 44.4759)',
       'POINT (-87.6298 41.8781)', 'POINT (-74.0059 40.7128)',
       'POINT (-0.07868 51.5048)', 'POINT (18.0686 59.3293)',
       'POINT (24.941 60.1733)', 'POINT (-115.14 36.1699)',
       'POINT (-118.244 34.0522)', 'POINT (-90.0715 29.9511)',
       'POINT (10.7522 59.9139)', 'POINT (-111.498 40.6461)',
       'POINT (-122.682 45.52)', 'POINT (-122.332 47.6062)',
       'POINT (-9.30148 38.7637)', 'POINT (-123.114 49.2612)',
       'POINT (12.5683 55.6761)', 'POINT (-1.51217 52.4066)',
       'POINT (-3.17909 51.4816)', 'POINT (-4.21278 55.8593)',
       'POINT (-77.0369 38.9072)', 'POINT (-95.3694 29.7602)',
       'POINT (144.963 -37.8141)', 'POINT (12.2858 41.53)',
       'POINT (139.41 35.41)', 'POINT (120.344 22.6297)',
       'POINT (139.638 35.4437)', 'POINT (151.207 -33.8675)',
       'POINT (28.0473 -26.2041)', 'POINT (-79.3832 43.6532)',
       '

In [376]:
cities_disclosure.rename(columns={'City_Loc':'geometry'},inplace=True)

In [377]:
cities_disclosure.loc[cities_disclosure.geometry=='POINT (2.1775 41.3823)']

,Reporting_Year,Account_Num,Org,City,Country,CDP_Reg,Reporting_Auth,Access,First_Time_Disc,Pop,Pop_Year,geometry,Last_Update
5,2020,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,1636762.0,2019.0,POINT (2.1775 41.3823),2020-09-07T09:45:19.793
6,2019,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,1620343.0,2018.0,POINT (2.1775 41.3823),2020-06-25T04:52:51.327
7,2018,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,"C40,CDP Cities",public,No,1604555.0,2015.0,POINT (2.1775 41.3823),2020-06-25T04:52:49.050


In [428]:
#remove null geopoints from the dataset
CD_df = cities_disclosure.dropna()

In [430]:
#add Lat Long coords
CD_df['lat']=CD_df.geometry.apply(lambda x: x.split(' ')[2].strip(')'))
CD_df['long']=CD_df.geometry.apply(lambda x: x.split(' ')[1].strip('('))

In [443]:
#Create a map
map1 = folium.Map(
    tiles='cartodbpositron',
    zoom_start=12)

CD_df.apply(lambda row:folium.CircleMarker(radius=3,location=[row["lat"], row["long"]],fill=False,color='blue').add_to(map1),axis=1)

map1

# Goal(s)

- Use Answers for Question IDs to find clusters?
- Use Answers to predict Country?

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [480]:
create a new df with country
df_with_country= pd.merge(df,CD_df[['Account_Num','Country']],on='Account_Num')

In [484]:
X= df_with_country.drop(columns='Country')
y= df_with_country['Country'].values

In [485]:
X_Train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25,random_state=0)

In [492]:
list(y_train)

['Brazil',
 'Italy',
 'United States of America',
 'Argentina',
 'Peru',
 'Denmark',
 'Portugal',
 'United States of America',
 'Portugal',
 'United States of America',
 'United States of America',
 'Canada',
 'Mexico',
 'Netherlands',
 'Philippines',
 'United States of America',
 'Portugal',
 'Italy',
 'Singapore',
 'Brazil',
 'Denmark',
 'United States of America',
 'Jordan',
 'United States of America',
 'Norway',
 'Brazil',
 'Brazil',
 'United States of America',
 'Mexico',
 'United States of America',
 'Mexico',
 'United States of America',
 'Portugal',
 'Canada',
 'Guatemala',
 'United States of America',
 'Japan',
 'United States of America',
 'United States of America',
 'Italy',
 'Mexico',
 'Canada',
 'Indonesia',
 'United States of America',
 'United States of America',
 'Canada',
 'Canada',
 'Netherlands',
 'Ecuador',
 'Brazil',
 'Sweden',
 'Peru',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United Sta

In [ ]:
def Transform_Data(dataset):
    
    

In [515]:
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /Users/matt/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matt/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/matt/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [516]:
import nltk
textexample = "Avengers: Endgame is a 2019 American superhero film based on the Marvel Comics superhero team the Avengers, produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. The movie features an ensemble cast including Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, and others. (Source: wikipedia)."
sentences = nltk.sent_tokenize(textexample)

In [508]:
words = [nltk.word_tokenize(sent) for sent in sentences]
words

[['Avengers',
  ':',
  'Endgame',
  'is',
  'a',
  '2019',
  'American',
  'superhero',
  'film',
  'based',
  'on',
  'the',
  'Marvel',
  'Comics',
  'superhero',
  'team',
  'the',
  'Avengers',
  ',',
  'produced',
  'by',
  'Marvel',
  'Studios',
  'and',
  'distributed',
  'by',
  'Walt',
  'Disney',
  'Studios',
  'Motion',
  'Pictures',
  '.'],
 ['The',
  'movie',
  'features',
  'an',
  'ensemble',
  'cast',
  'including',
  'Robert',
  'Downey',
  'Jr.',
  ',',
  'Chris',
  'Evans',
  ',',
  'Mark',
  'Ruffalo',
  ',',
  'Chris',
  'Hemsworth',
  ',',
  'and',
  'others',
  '.'],
 ['(', 'Source', ':', 'wikipedia', ')', '.']]

In [512]:
scores = [nltk.pos_tag(w) for w in words]
scores

[[('Avengers', 'NNS'),
  (':', ':'),
  ('Endgame', 'NN'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('2019', 'JJ'),
  ('American', 'JJ'),
  ('superhero', 'NN'),
  ('film', 'NN'),
  ('based', 'VBN'),
  ('on', 'IN'),
  ('the', 'DT'),
  ('Marvel', 'NNP'),
  ('Comics', 'NNP'),
  ('superhero', 'NN'),
  ('team', 'NN'),
  ('the', 'DT'),
  ('Avengers', 'NNPS'),
  (',', ','),
  ('produced', 'VBN'),
  ('by', 'IN'),
  ('Marvel', 'NNP'),
  ('Studios', 'NNP'),
  ('and', 'CC'),
  ('distributed', 'VBN'),
  ('by', 'IN'),
  ('Walt', 'NNP'),
  ('Disney', 'NNP'),
  ('Studios', 'NNP'),
  ('Motion', 'NNP'),
  ('Pictures', 'NNP'),
  ('.', '.')],
 [('The', 'DT'),
  ('movie', 'NN'),
  ('features', 'VBZ'),
  ('an', 'DT'),
  ('ensemble', 'JJ'),
  ('cast', 'NN'),
  ('including', 'VBG'),
  ('Robert', 'NNP'),
  ('Downey', 'NNP'),
  ('Jr.', 'NNP'),
  (',', ','),
  ('Chris', 'NNP'),
  ('Evans', 'NNP'),
  (',', ','),
  ('Mark', 'NNP'),
  ('Ruffalo', 'NNP'),
  (',', ','),
  ('Chris', 'NNP'),
  ('Hemsworth', 'NNP'),
  (',', ',

In [529]:
def process(text):
    tok_words = nltk.word_tokenize(text)
    POS_words = nltk.pos_tag(tok_words)
    return POS_words
processed_text= process(textexample)
processed_text

[('Avengers', 'NNS'),
 (':', ':'),
 ('Endgame', 'NN'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('2019', 'JJ'),
 ('American', 'JJ'),
 ('superhero', 'NN'),
 ('film', 'NN'),
 ('based', 'VBN'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('Marvel', 'NNP'),
 ('Comics', 'NNP'),
 ('superhero', 'NN'),
 ('team', 'NN'),
 ('the', 'DT'),
 ('Avengers', 'NNPS'),
 (',', ','),
 ('produced', 'VBN'),
 ('by', 'IN'),
 ('Marvel', 'NNP'),
 ('Studios', 'NNP'),
 ('and', 'CC'),
 ('distributed', 'VBN'),
 ('by', 'IN'),
 ('Walt', 'NNP'),
 ('Disney', 'NNP'),
 ('Studios', 'NNP'),
 ('Motion', 'NNP'),
 ('Pictures', 'NNP'),
 ('.', '.'),
 ('The', 'DT'),
 ('movie', 'NN'),
 ('features', 'VBZ'),
 ('an', 'DT'),
 ('ensemble', 'JJ'),
 ('cast', 'NN'),
 ('including', 'VBG'),
 ('Robert', 'NNP'),
 ('Downey', 'NNP'),
 ('Jr.', 'NNP'),
 (',', ','),
 ('Chris', 'NNP'),
 ('Evans', 'NNP'),
 (',', ','),
 ('Mark', 'NNP'),
 ('Ruffalo', 'NNP'),
 (',', ','),
 ('Chris', 'NNP'),
 ('Hemsworth', 'NNP'),
 (',', ','),
 ('and', 'CC'),
 ('others', 'NNS'),
 ('.', '.'),


In [531]:
nltk.ne_chunk?

In [530]:
chunks = nltk.ne_chunk(processed_text)
print(chunks)


(S
  Avengers/NNS
  :/:
  Endgame/NN
  is/VBZ
  a/DT
  2019/JJ
  (GPE American/JJ)
  superhero/NN
  film/NN
  based/VBN
  on/IN
  the/DT
  (ORGANIZATION Marvel/NNP Comics/NNP)
  superhero/NN
  team/NN
  the/DT
  (ORGANIZATION Avengers/NNPS)
  ,/,
  produced/VBN
  by/IN
  (PERSON Marvel/NNP Studios/NNP)
  and/CC
  distributed/VBN
  by/IN
  (PERSON Walt/NNP Disney/NNP Studios/NNP)
  Motion/NNP
  Pictures/NNP
  ./.
  The/DT
  movie/NN
  features/VBZ
  an/DT
  ensemble/JJ
  cast/NN
  including/VBG
  (PERSON Robert/NNP Downey/NNP Jr./NNP)
  ,/,
  (PERSON Chris/NNP Evans/NNP)
  ,/,
  (PERSON Mark/NNP Ruffalo/NNP)
  ,/,
  (PERSON Chris/NNP Hemsworth/NNP)
  ,/,
  and/CC
  others/NNS
  ./.
  (/(
  (PERSON Source/NN)
  :/:
  wikipedia/NN
  )/)
  ./.)
